<a href="https://colab.research.google.com/github/chominyeong/2023_Manufacturing-data_Competition/blob/main/%EB%93%B1%EA%B0%84%EA%B2%A9%EB%B6%84%ED%95%A0_%EC%A7%9D%EB%A7%9E%EC%B6%A4%ED%91%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. 데이터 및 라이브러리 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import math
import random
import scipy.stats as stats
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error

pd.set_option('display.max_columns', None)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/2023 제조 빅데이터 분석 경진대회/OBD_data.csv')
df = df.iloc[:1296, :]

# 1. 등간격분할방식
각 범주의 치수의 폭이 동일하도록 범주를 분류하는 방식

1. 정규분포에서 범위 : 0.0013
2. Bgrade 구할 때 : 평균(mean) 사용
3. tube와 shaft의 범주의 수 : 10*8 → 2nd 90% 이상인 최소의 범주

## Tube=10, Shaft=8

In [ ]:
Tnum = 10
Snum = 8

np.random.seed(1906)
Tdata=np.random.normal(16.5212, 0.0189, (100000, 1))
Sdata=np.random.normal(24.8696, 0.0072, (100000, 1))
simulDF = pd.DataFrame(np.concatenate((Tdata,Sdata),axis=1),columns=["Tube","Shaft"])
simulDF['BallSize']=12.0385+0.2072*simulDF['Tube']-0.4534	*simulDF['Shaft']
simulDF['BallSize_descrete']=(simulDF['BallSize']*500).round() * 0.002

rv1 = stats.norm(16.5212,0.0189)
rv2 = stats.norm(24.8696,0.0072)
reject=0.0013
Tbins = [-np.inf]+[rv1.ppf(reject)+(rv1.ppf(1-reject)-rv1.ppf(reject))*k/Tnum for k in range(1,Tnum)]+[np.inf]
Sbins = [-np.inf]+[rv2.ppf(reject)+(rv2.ppf(1-reject)-rv2.ppf(reject))*k/Snum for k in range(1,Snum)]+[np.inf]

labels_tube = [chr(65 + i) for i in range(Tnum)]
labels_shaft = [chr(65 + i) for i in range(Snum)]

simulDF['Tgroup'] = pd.cut(simulDF['Tube'], bins=Tbins, labels=labels_tube)
simulDF['Sgroup'] = pd.cut(simulDF['Shaft'], bins=Sbins, labels=labels_shaft)

tube_grouped_mean = simulDF.groupby('Tgroup')['Tube'].transform('mean')
shaft_grouped_mean = simulDF.groupby('Sgroup')['Shaft'].transform('mean')

simulDF['Bgrade'] = 12.0385 + 0.2072 * tube_grouped_mean - 0.4534 * shaft_grouped_mean
simulDF['Bgrade'] = (simulDF['Bgrade'] / 0.002).round() * 0.002

mse = mean_squared_error(simulDF['BallSize'], simulDF['Bgrade'])
accuracy_exact = sum(simulDF['BallSize_descrete'] == simulDF['Bgrade'])/100000
accuracy_within_002 = sum(abs(simulDF['BallSize_descrete']-simulDF['Bgrade']) <= 0.002)/100000

# 결과 출력
print(f"Tube Bins: {Tnum}, Shaft Bins: {Snum}\n", simulDF[['Tube', 'Shaft', 'Tgroup', 'Sgroup', 'Bgrade']], "\n")
print(f"Mean Squared Error: {mse}")
print(f"Accuracy (Exact Match) = 1st 조합: {accuracy_exact}")
print(f"Accuracy (Within 0.002) = 2nd 조합: {accuracy_within_002}")
print("\n")

Tube Bins: 10, Shaft Bins: 8
             Tube      Shaft Tgroup Sgroup  Bgrade
0      16.542444  24.872523      G      E   4.188
1      16.517322  24.873397      E      E   4.184
2      16.507366  24.873938      D      E   4.182
3      16.524452  24.873036      F      E   4.186
4      16.543780  24.862423      G      C   4.192
...          ...        ...    ...    ...     ...
99995  16.495444  24.877294      C      F   4.176
99996  16.524871  24.862153      F      C   4.190
99997  16.503999  24.876274      D      F   4.178
99998  16.558775  24.871734      I      E   4.192
99999  16.508574  24.854177      D      B   4.188

[100000 rows x 5 columns] 

Mean Squared Error: 1.2486947958962905e-06
Accuracy (Exact Match) = 1st 조합: 0.61553
Accuracy (Within 0.002) = 2nd 조합: 0.90049




In [ ]:
simulDF

,Tube,Shaft,BallSize,BallSize_descrete,Tgroup,Sgroup,Bgrade
0,16.542444,24.872523,4.188893,4.188,G,E,4.188
1,16.517322,24.873397,4.183291,4.184,E,E,4.184
2,16.507366,24.873938,4.180983,4.180,D,E,4.182
3,16.524452,24.873036,4.184932,4.184,F,E,4.186
4,16.543780,24.862423,4.193749,4.194,G,C,4.192
...,...,...,...,...,...,...,...
99995,16.495444,24.877294,4.176991,4.176,C,F,4.176
99996,16.524871,24.862153,4.189953,4.190,F,C,4.190
99997,16.503999,24.876274,4.179226,4.180,D,F,4.178
99998,16.558775,24.871734,4.192634,4.192,I,E,4.192


In [ ]:
grade_df = simulDF[['Tgroup', 'Sgroup', 'Bgrade']]
grade_df

,Tgroup,Sgroup,Bgrade
0,G,E,4.188
1,E,E,4.184
2,D,E,4.182
3,F,E,4.186
4,G,C,4.192
...,...,...,...
99995,C,F,4.176
99996,F,C,4.190
99997,D,F,4.178
99998,I,E,4.192


In [ ]:
# pd.set_option('display.max_rows', None)
grade_df.groupby(['Tgroup', 'Sgroup']).mean()

Bgrade
Tgroup Sgroup        
A      A        4.184
       B        4.182
       C        4.178
       D        4.176
       E        4.174
...               ...
J      D        4.198
       E        4.196
       F        4.194
       G        4.190
       H        4.188

[80 rows x 1 columns]

In [ ]:
grade_df.groupby(['Tgroup', 'Sgroup']).mean().isnull().sum()

Bgrade    0
dtype: int64

### 1) Tube grade (10개)

In [ ]:
Tbins

[-inf,
 16.475666819171483,
 16.48705011437861,
 16.498433409585743,
 16.509816704792872,
 16.5212,
 16.53258329520713,
 16.543966590414257,
 16.55534988562139,
 16.566733180828518,
 inf]

In [ ]:
def inner(x):
  if x < 16.475666819171483: return 'A'
  elif x >= 16.475666819171483 and x <= 16.48705011437861: return 'B'
  elif x >= 16.48705011437861 and x <= 16.498433409585743: return 'C'
  elif x >= 16.498433409585743 and x <= 16.509816704792872: return 'D'
  elif x >= 16.509816704792872 and x <= 16.5212: return 'E'
  elif x >= 16.5212 and x <= 16.53258329520713: return 'F'
  elif x >= 16.53258329520713 and x <= 16.543966590414257: return 'G'
  elif x >= 16.543966590414257 and x <= 16.55534988562139: return 'H'
  elif x >= 16.55534988562139 and x <= 16.566733180828518: return 'I'
  elif x > 16.566733180828518: return 'J'

### 2) Shaft grade (8개)

In [ ]:
Sbins

[-inf,
 24.8533381497041,
 24.8587587664694,
 24.8641793832347,
 24.8696,
 24.875020616765298,
 24.880441233530597,
 24.885861850295896,
 inf]

In [ ]:
def outer(x):
  if x < 24.8533381497041: return 'A'
  elif x >= 24.8533381497041 and x <= 24.8587587664694: return 'B'
  elif x >= 24.8587587664694 and x <= 24.8641793832347: return 'C'
  elif x >= 24.8641793832347 and x <= 24.8696: return 'D'
  elif x >= 24.8696 and x <= 24.875020616765298: return 'E'
  elif x >= 24.875020616765298 and x <= 24.880441233530597: return 'F'
  elif x >= 24.880441233530597 and x <= 24.885861850295896: return 'G'
  elif x > 24.885861850295896: return 'H'

### 1) Ball grade (10 * 8개)

In [ ]:
pd.set_option('display.max_rows', None)
grade_df.groupby(['Tgroup', 'Sgroup']).mean()

Bgrade
Tgroup Sgroup        
A      A        4.184
       B        4.182
       C        4.178
       D        4.176
       E        4.174
       F        4.172
       G        4.170
       H        4.166
B      A        4.186
       B        4.184
       C        4.182
       D        4.178
       E        4.176
       F        4.174
       G        4.172
       H        4.170
C      A        4.188
       B        4.186
       C        4.184
       D        4.182
       E        4.178
       F        4.176
       G        4.174
       H        4.172
D      A        4.190
       B        4.188
       C        4.186
       D        4.184
       E        4.182
       F        4.178
       G        4.176
       H        4.174
E      A        4.194
       B        4.190
       C        4.188
       D        4.186
       E        4.184
       F        4.182
       G        4.178
       H        4.176
F      A        4.196
       B        4.192
       C        4.190
       D        4.188
       E        4.186
       F        4.184
       G        4.182
       H        4.178
G      A        4.198
       B        4.196
       C        4.192
       D        4.190
       E        4.188
       F        4.186
       G        4.184
       H        4.180
H      A        4.200
       B        4.198
       C        4.196
       D        4.192
       E        4.190
       F        4.188
       G        4.186
       H        4.184
I      A        4.202
       B        4.200
       C        4.198
       D        4.196
       E        4.192
       F        4.190
       G        4.188
       H        4.186
J      A        4.204
       B        4.202
       C        4.200
       D        4.198
       E        4.196
       F        4.194
       G        4.190
       H        4.188

In [ ]:
def ballsize(tube, shaft):
 if (tube == 'A')&(shaft == 'A'): return 4.184
 elif (tube == 'A')&(shaft == 'B'): return 4.182
 elif (tube == 'A')&(shaft == 'C'): return 4.178
 elif (tube == 'A')&(shaft == 'D'): return 4.176
 elif (tube == 'A')&(shaft == 'E'): return 4.174
 elif (tube == 'A')&(shaft == 'F'): return 4.172
 elif (tube == 'A')&(shaft == 'G'): return 4.170
 elif (tube == 'A')&(shaft == 'H'): return 4.166

 elif (tube == 'B')&(shaft == 'A'): return 4.186
 elif (tube == 'B')&(shaft == 'B'): return 4.184
 elif (tube == 'B')&(shaft == 'C'): return 4.182
 elif (tube == 'B')&(shaft == 'D'): return 4.178
 elif (tube == 'B')&(shaft == 'E'): return 4.176
 elif (tube == 'B')&(shaft == 'F'): return 4.174
 elif (tube == 'B')&(shaft == 'G'): return 4.172
 elif (tube == 'B')&(shaft == 'H'): return 4.170

 elif (tube == 'C')&(shaft == 'A'): return 4.188
 elif (tube == 'C')&(shaft == 'B'): return 4.186
 elif (tube == 'C')&(shaft == 'C'): return 4.184
 elif (tube == 'C')&(shaft == 'D'): return 4.182
 elif (tube == 'C')&(shaft == 'E'): return 4.178
 elif (tube == 'C')&(shaft == 'F'): return 4.176
 elif (tube == 'C')&(shaft == 'G'): return 4.174
 elif (tube == 'C')&(shaft == 'H'): return 4.172

 elif (tube == 'D')&(shaft == 'A'): return 4.190
 elif (tube == 'D')&(shaft == 'B'): return 4.188
 elif (tube == 'D')&(shaft == 'C'): return 4.186
 elif (tube == 'D')&(shaft == 'D'): return 4.184
 elif (tube == 'D')&(shaft == 'E'): return 4.182
 elif (tube == 'D')&(shaft == 'F'): return 4.178
 elif (tube == 'D')&(shaft == 'G'): return 4.176
 elif (tube == 'D')&(shaft == 'H'): return 4.174

 elif (tube == 'E')&(shaft == 'A'): return 4.194
 elif (tube == 'E')&(shaft == 'B'): return 4.190
 elif (tube == 'E')&(shaft == 'C'): return 4.188
 elif (tube == 'E')&(shaft == 'D'): return 4.186
 elif (tube == 'E')&(shaft == 'E'): return 4.184
 elif (tube == 'E')&(shaft == 'F'): return 4.182
 elif (tube == 'E')&(shaft == 'G'): return 4.178
 elif (tube == 'E')&(shaft == 'H'): return 4.176

 elif (tube == 'F')&(shaft == 'A'): return 4.196
 elif (tube == 'F')&(shaft == 'B'): return 4.192
 elif (tube == 'F')&(shaft == 'C'): return 4.190
 elif (tube == 'F')&(shaft == 'D'): return 4.188
 elif (tube == 'F')&(shaft == 'E'): return 4.186
 elif (tube == 'F')&(shaft == 'F'): return 4.184
 elif (tube == 'F')&(shaft == 'G'): return 4.182
 elif (tube == 'F')&(shaft == 'H'): return 4.178

 elif (tube == 'G')&(shaft == 'A'): return 4.198
 elif (tube == 'G')&(shaft == 'B'): return 4.196
 elif (tube == 'G')&(shaft == 'C'): return 4.192
 elif (tube == 'G')&(shaft == 'D'): return 4.190
 elif (tube == 'G')&(shaft == 'E'): return 4.188
 elif (tube == 'G')&(shaft == 'F'): return 4.186
 elif (tube == 'G')&(shaft == 'G'): return 4.184
 elif (tube == 'G')&(shaft == 'H'): return 4.180

 elif (tube == 'H')&(shaft == 'A'): return 4.200
 elif (tube == 'H')&(shaft == 'B'): return 4.198
 elif (tube == 'H')&(shaft == 'C'): return 4.196
 elif (tube == 'H')&(shaft == 'D'): return 4.192
 elif (tube == 'H')&(shaft == 'E'): return 4.190
 elif (tube == 'H')&(shaft == 'F'): return 4.188
 elif (tube == 'H')&(shaft == 'G'): return 4.186
 elif (tube == 'H')&(shaft == 'H'): return 4.184

 elif (tube == 'I')&(shaft == 'A'): return 4.202
 elif (tube == 'I')&(shaft == 'B'): return 4.200
 elif (tube == 'I')&(shaft == 'C'): return 4.198
 elif (tube == 'I')&(shaft == 'D'): return 4.196
 elif (tube == 'I')&(shaft == 'E'): return 4.192
 elif (tube == 'I')&(shaft == 'F'): return 4.190
 elif (tube == 'I')&(shaft == 'G'): return 4.188
 elif (tube == 'I')&(shaft == 'H'): return 4.186

 elif (tube == 'J')&(shaft == 'A'): return 4.204
 elif (tube == 'J')&(shaft == 'B'): return 4.202
 elif (tube == 'J')&(shaft == 'C'): return 4.200
 elif (tube == 'J')&(shaft == 'D'): return 4.198
 elif (tube == 'J')&(shaft == 'E'): return 4.196
 elif (tube == 'J')&(shaft == 'F'): return 4.194
 elif (tube == 'J')&(shaft == 'G'): return 4.190
 elif (tube == 'J')&(shaft == 'H'): return 4.188

# 2. validation data

- TC1 ~ TC3 (M 지점에서의 inner 측정값 1, 2, 3)의 평균값을 Tube의 측정값으로
- SB1 ~ SB3(M 지점에서의 outer 측정값 1, 2, 3)의 평균값을 Shaft의 측정값으로

In [ ]:
valid_df = pd.read_csv('/content/drive/MyDrive/2023 제조 빅데이터 분석 경진대회/validation_OBD_data.csv')
valid_df.shape

(50, 21)

In [ ]:
valid_df = pd.read_csv('/content/drive/MyDrive/2023 제조 빅데이터 분석 경진대회/validation_OBD_data.csv')
valid_df['Tmean'] = (valid_df['TC1'] + valid_df['TC2'] + valid_df['TC3'])/3
valid_df['Smean'] = (valid_df['SB1'] + valid_df['SB2'] + valid_df['SB3'])/3
valid_df = valid_df[['TubeNo', 'ShaftNo', 'Tmean', 'Smean']]
valid_df

In [ ]:
valid_df['Tgroup'] = valid_df['Tmean'].apply(inner)
valid_df['Sgroup'] = valid_df['Smean'].apply(outer)
valid_df

In [ ]:
valid_df['BallSize'] = valid_df.apply(lambda row: ballsize(row['Tgroup'], row['Sgroup']), axis=1)
valid_df

In [ ]:
valid_df.isnull().sum()

TubeNo      0
ShaftNo     0
Tmean       0
Smean       0
Tgroup      0
Sgroup      0
BallSize    0
dtype: int64

In [ ]:
# pd.set_option('display.max_rows', None)
valid_df.groupby(['Tgroup', 'Sgroup']).count()

,,TubeNo,ShaftNo,Tmean,Smean,BallSize
Tgroup,Sgroup,,,,,
C,H,1,1,1,1,1
D,H,3,3,3,3,3
E,H,5,5,5,5,5
F,H,12,12,12,12,12
G,H,9,9,9,9,9
H,H,10,10,10,10,10
I,H,8,8,8,8,8
J,H,2,2,2,2,2


In [ ]:
valid_df = valid_df[['TubeNo', 'ShaftNo', 'BallSize']]
valid_df

In [ ]:
valid_df.to_csv('/content/drive/MyDrive/2023 제조 빅데이터 분석 경진대회/valid_df.csv')

# 3. 짝맞춤표

In [ ]:
grade_standard = grade_df.groupby(['Tgroup', 'Sgroup']).mean()
grade_standard

Bgrade
Tgroup Sgroup        
A      A        4.184
       B        4.182
       C        4.178
       D        4.176
       E        4.174
       F        4.172
       G        4.170
       H        4.166
B      A        4.186
       B        4.184
       C        4.182
       D        4.178
       E        4.176
       F        4.174
       G        4.172
       H        4.170
C      A        4.188
       B        4.186
       C        4.184
       D        4.182
       E        4.178
       F        4.176
       G        4.174
       H        4.172
D      A        4.190
       B        4.188
       C        4.186
       D        4.184
       E        4.182
       F        4.178
       G        4.176
       H        4.174
E      A        4.194
       B        4.190
       C        4.188
       D        4.186
       E        4.184
       F        4.182
       G        4.178
       H        4.176
F      A        4.196
       B        4.192
       C        4.190
       D        4.188
       E        4.186
       F        4.184
       G        4.182
       H        4.178
G      A        4.198
       B        4.196
       C        4.192
       D        4.190
       E        4.188
       F        4.186
       G        4.184
       H        4.180
H      A        4.200
       B        4.198
       C        4.196
       D        4.192
       E        4.190
       F        4.188
       G        4.186
       H        4.184
I      A        4.202
       B        4.200
       C        4.198
       D        4.196
       E        4.192
       F        4.190
       G        4.188
       H        4.186
J      A        4.204
       B        4.202
       C        4.200
       D        4.198
       E        4.196
       F        4.194
       G        4.190
       H        4.188

In [ ]:
grade_standard['2nd (-0.002)'] = grade_standard['Bgrade'] - 0.002
grade_standard['2nd (+0.002)'] = grade_standard['Bgrade'] + 0.002
grade_standard = grade_standard[['2nd (-0.002)', 'Bgrade', '2nd (+0.002)']]
grade_standard.columns = ['2nd (-0.002)', '1st', '2nd (+0.002)']
grade_standard

2nd (-0.002)    1st  2nd (+0.002)
Tgroup Sgroup                                   
A      A              4.182  4.184         4.186
       B              4.180  4.182         4.184
       C              4.176  4.178         4.180
       D              4.174  4.176         4.178
       E              4.172  4.174         4.176
       F              4.170  4.172         4.174
       G              4.168  4.170         4.172
       H              4.164  4.166         4.168
B      A              4.184  4.186         4.188
       B              4.182  4.184         4.186
       C              4.180  4.182         4.184
       D              4.176  4.178         4.180
       E              4.174  4.176         4.178
       F              4.172  4.174         4.176
       G              4.170  4.172         4.174
       H              4.168  4.170         4.172
C      A              4.186  4.188         4.190
       B              4.184  4.186         4.188
       C              4.182  4.184         4.186
       D              4.180  4.182         4.184
       E              4.176  4.178         4.180
       F              4.174  4.176         4.178
       G              4.172  4.174         4.176
       H              4.170  4.172         4.174
D      A              4.188  4.190         4.192
       B              4.186  4.188         4.190
       C              4.184  4.186         4.188
       D              4.182  4.184         4.186
       E              4.180  4.182         4.184
       F              4.176  4.178         4.180
       G              4.174  4.176         4.178
       H              4.172  4.174         4.176
E      A              4.192  4.194         4.196
       B              4.188  4.190         4.192
       C              4.186  4.188         4.190
       D              4.184  4.186         4.188
       E              4.182  4.184         4.186
       F              4.180  4.182         4.184
       G              4.176  4.178         4.180
       H              4.174  4.176         4.178
F      A              4.194  4.196         4.198
       B              4.190  4.192         4.194
       C              4.188  4.190         4.192
       D              4.186  4.188         4.190
       E              4.184  4.186         4.188
       F              4.182  4.184         4.186
       G              4.180  4.182         4.184
       H              4.176  4.178         4.180
G      A              4.196  4.198         4.200
       B              4.194  4.196         4.198
       C              4.190  4.192         4.194
       D              4.188  4.190         4.192
       E              4.186  4.188         4.190
       F              4.184  4.186         4.188
       G              4.182  4.184         4.186
       H              4.178  4.180         4.182
H      A              4.198  4.200         4.202
       B              4.196  4.198         4.200
       C              4.194  4.196         4.198
       D              4.190  4.192         4.194
       E              4.188  4.190         4.192
       F              4.186  4.188         4.190
       G              4.184  4.186         4.188
       H              4.182  4.184         4.186
I      A              4.200  4.202         4.204
       B              4.198  4.200         4.202
       C              4.196  4.198         4.200
       D              4.194  4.196         4.198
       E              4.190  4.192         4.194
       F              4.188  4.190         4.192
       G              4.186  4.188         4.190
       H              4.184  4.186         4.188
J      A              4.202  4.204         4.206
       B              4.200  4.202         4.204
       C              4.198  4.200         4.202
       D              4.196  4.198         4.200
       E              4.194  4.196         4.198
       F              4.192  4.194         4.196
       G              4.188  4.190         4.192
       H              4.186  4.188         4.1

In [ ]:
# grade_standard.to_csv('/content/drive/MyDrive/2023 제조 빅데이터 분석 경진대회/짝맞춤표.csv')